In [ ]:
import yfinance
import json
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_mainBusiness(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Main Business of {company_name}")

def get_description(inputs):
    wiki = WikipediaAPIWrapper()
    company_name = inputs["company_name"]
    return wiki.run(f"Main Business of {company_name}")


functions_map = {
    "get_ticker": get_ticker,
    "get_mainBusiness":get_mainBusiness,
    "get_description": get_description,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given a ticker symbol (i.e AAPL) returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                 "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_description",
            "description": "Given a ticker symbol (i.e AAPL) returns its description",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                 "required": ["company_name"],
            },
        },
    },
        {
        "type": "function",
        "function": {
            "name": "get_mainBusiness",
            "description": "Given a ticker symbol (i.e AAPL) returns its main business",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                 "required": ["company_name"],
            },
        },
    },
]

In [ ]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You are an investment consultant. You need to identify promising companies.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )

assistant_id = "asst_VD637NbTusuC1egAhwdKeZuB"

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know about Salesforce",
        }
    ]
)
thread

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

In [ ]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    with open("output.txt", "w") as file:
        for message in messages:
            file.write(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    output = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=output
    )

In [ ]:
submit_tool_outputs(run.id, thread.id)

In [ ]:
run = get_run(run.id, thread.id)
run.status

In [ ]:
get_messages(thread.id)